# Minimal example of forecasting using ARIMA

Note that you are expected to do substantially more than what is here in yor project.
This is just an example to get you started.

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

In [ ]:
DATA_DIR = "~/Dropbox/Dropbox_project_data/MDS_examples"

Load the data

In [ ]:
df_tles = pd.read_csv(
    os.path.join(DATA_DIR, "Sentinel-3A.csv"),
    index_col=0, 
    parse_dates=True
)

In [ ]:
df_tles

Check that the index has been loaded as a datetime object

In [ ]:
df_tles.index.inferred_type

Make our life easier by making a dataframe with only the Brouwer mean motion, then centering and rescaling the data. Note that the rescaling is only done to make the numbers easier to read on screen (it has no effect on the operation of ARIMA).

In [ ]:
df_mm = df_tles[["Brouwer mean motion"]]
df_mm = (df_mm - df_mm.mean())*1e7

In [ ]:
#sns.lineplot(df_mm["2020-01-01":"2022-06-01"])
sns.lineplot(df_mm)

In [ ]:
df_mm

The next couple of cells resample the data, with linear interoplation, so that it becomes a regularly-sampled time series. This is split out into a few cells so that you can see what is happening.

In [ ]:
df_mm.index = pd.to_datetime(df_mm.index)
start_time = df_mm.index.min()
end_time = df_mm.index.max()
regular_index = pd.date_range(start=start_time, end=end_time, freq='1D')
regular_index  

In [ ]:
df_mm_resampled = df_mm.reindex(df_mm.index.union(regular_index)).sort_index()
df_mm_resampled

In [ ]:
df_mm_resampled = df_mm_resampled.interpolate(method='linear')
df_mm_resampled

In [ ]:
df_mm_resampled = df_mm_resampled.loc[regular_index]
df_mm_resampled

We then plot the resampled data against the original data as a sanity check

In [ ]:
fig, ax = plt.subplots()
df_mm_resampled.loc["2019-01"]["Brouwer mean motion"].plot(ax=ax, label="Interpolated")
df_mm.loc["2019-01"]["Brouwer mean motion"].plot(ax=ax, label="Original")
ax.legend()

We then have a look at the ACF and PACF to aid in model selection

In [ ]:
# plot an ACF
sm.graphics.tsa.plot_acf(df_mm_resampled["Brouwer mean motion"], lags=100)
plt.show()

In [ ]:
# plot an ACF
sm.graphics.tsa.plot_pacf(df_mm_resampled["Brouwer mean motion"], lags=30)
plt.show()

Do a test for stationarity

In [ ]:
adfuller(df_mm_resampled["Brouwer mean motion"])

We can fit a single AR model

In [ ]:
model = sm.tsa.ARIMA(df_mm_resampled, order=(2, 0, 0))
results = model.fit()
results.summary()

Now do a grid search for a better model

In [ ]:
AR_MAX_ORDER = 5
MA_MAX_ORDER = 5
MAX_DIFF = 0

best_aic = float("inf")
best_params = None
best_model = None
for p in range(AR_MAX_ORDER + 1):
    for d in range(MAX_DIFF + 1):
        for q in range(MA_MAX_ORDER + 1):            
            try:
                model = sm.tsa.ARIMA(df_mm_resampled, order=(p, d, q))
                results = model.fit()
                print(f"ARIMA({p}, {d}, {q}) AIC: {results.aic}")
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_params = (p, d, q)
                    best_model = results
            except:
                continue
print(f"Best ARIMA model: {best_params} with AIC: {best_aic}")

In [ ]:
residuals = best_model.resid
fig, ax = plt.subplots()
residuals.plot(ax=ax)

In [ ]:
# plot the ACF of the residuals
sm.graphics.tsa.plot_acf(residuals, lags=30)
plt.show()

In [ ]:
# plot the PACF of the residuals
sm.graphics.tsa.plot_pacf(residuals, lags=30)
plt.show()

In [ ]:
# plot a q-q plot of the residuals
sm.qqplot(residuals, line='s')
plt.show()